Psuedocode

In [ ]:
# Inputs from reader/model each decision tick
state = {
  "confluence": ĉ,                 # 0..1
  "growth_rate": ĝ,                # 1/hr
  "stress": σ,                     # a.u.
  "viability": ν,                  # 0..100%
  "time_since_last_passage": τ,    # hr
  "forecast_confluence_2h": ĉ2h,
  "forecast_confluence_4h": ĉ4h
}

# Global thresholds
TH = {
  "crowd_now": 0.85,
  "prepare_soon": 0.75,
  "min_tau_hr": 12,
  "stress_sigma": 2.0
}

def cohort(state):
    ĉ, ĝ, σ, ν, τ, ĉ2h, ĉ4h = (state[k] for k in
        ["confluence","growth_rate","stress","viability",
         "time_since_last_passage","forecast_confluence_2h","forecast_confluence_4h"])
    if σ > baseline_stress + TH["stress_sigma"]*stress_sd or (ν is not None and ν < 90):
        return "S"       # stress-flag cohort
    if ĉ < 0.50 and ĝ > 0:
        return "G0"
    if 0.50 <= ĉ < 0.70 or ĉ4h >= TH["prepare_soon"]:
        return "G1"
    if 0.70 <= ĉ < TH["crowd_now"] or ĉ2h >= TH["crowd_now"] or d2_dt2_density < 0:
        return "G2"
    if ĉ >= TH["crowd_now"]:
        return "G3"
    return "G1"

def split_decision(state):
    group = cohort(state)
    τ = state["time_since_last_passage"]
    decision = {"action":"hold","why":group,"mix":{}, "media":{}, "dilution":None, "when":"now"}

    # safety overrides
    if group == "S":
        decision.update({
          "action":"care",
          "mix":{"waveform":"step","frequency_hz":0.4,"cycles":4,"pipette_speed_ul_s":80},
          "media":{"exchange_pct":20,"buffer_mM":"+10"},
          "when":"now"
        })
        return decision

    if group in ["G0","G1"]:
        decision.update({
          "action":"maintain",
          "mix":{"waveform":"sinusoid","frequency_hz":0.5 if group=="G0" else 0.7,
                 "cycles":3 if group=="G0" else 5, "pipette_speed_ul_s":90},
          "media":{"exchange_pct":10 if group=="G0" else 20}
        })
        return decision

    if group == "G2":
        # schedule soft split soon; stagger to smooth robot load
        decision.update({
          "action":"split_soft" if τ >= 8 else "maintain",
          "dilution":"1:1",
          "mix":{"waveform":"sinusoid","frequency_hz":0.9,"cycles":6,"pipette_speed_ul_s":100},
          "media":{"exchange_pct":30},
          "when":"T+2h" if τ >= 8 else "recheck_60min"
        })
        return decision

    if group == "G3":
        if τ < TH["min_tau_hr"]:
            # too soon; do care + hold
            decision.update({
              "action":"care",
              "mix":{"waveform":"step","frequency_hz":0.6,"cycles":5,"pipette_speed_ul_s":90},
              "media":{"exchange_pct":20},
              "when":"recheck_60min"
            })
        else:
            decision.update({
              "action":"split_standard",
              "dilution":"1:2",
              "mix":{"waveform":"triangular","frequency_hz":1.0,"cycles":7,"pipette_speed_ul_s":100},
              "media":{"exchange_pct":35},
              "when":"now"
            })
        return decision

# Batch scheduling (grouping wells)
def schedule_batches(well_states):
    # assign by cohort → create time slots to avoid crowding robot
    slots = {"now":[], "T+1h":[], "T+2h":[], "T+4h":[]}
    for well_id, st in well_states.items():
        d = split_decision(st)
        # stagger G2 soft splits into T+2h, G3 standard into 'now' and 'T+1h'
        if d["action"] == "split_standard":
            slots["now" if len(slots["now"]) < 16 else "T+1h"].append((well_id, d))
        elif d["action"] == "split_soft":
            slots["T+2h" if len(slots["T+2h"]) < 24 else "T+4h"].append((well_id, d))
        else:
            slots["now"].append((well_id, d))  # maintain/care ops run quickly
    return slots
